In [4]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time

In [5]:
def getdataloaders(data,train_batch_size,test_batch_size):
    np.random.shuffle(data.T)
    train_data1 = data[:,:200000]
    test_data1 = data[:,200000:]
    
    #creating train batches
    num_batches_train = int(train_data1.shape[1]/train_batch_size)
    train_data= np.empty([num_batches_train,train_data1.shape[0],train_batch_size])
    for idx in range (0, num_batches_train):
        train_data[idx,:,:]=train_data1[:,idx*train_batch_size:(idx + 1)*train_batch_size]
    
    #creating test batches
    num_batches_test = int(test_data1.shape[1]/test_batch_size)
    test_data =np.empty([num_batches_test,test_data1.shape[0],test_batch_size])
    for idx in range (0, num_batches_test):
        test_data[idx,:,:]=test_data1[:,idx*test_batch_size:(idx + 1)*test_batch_size]
    
    return train_data,test_data

In [16]:
class network(nn.Module):
    def __init__(self):
        super(network,self).__init__()
        input_nodes = 47
        hidden_nodes1 = 256
        hidden_nodes2 = 256
        output_nodes = 1
        
        self.fc1 = nn.Linear(input_nodes,hidden_nodes1)
        nn.init.xavier_uniform_(self.fc1.weight)
        self.bn1 = nn.BatchNorm1d(num_features=hidden_nodes1)
        
        self.fc2 = nn.Linear(hidden_nodes1,hidden_nodes2)
        nn.init.xavier_uniform_(self.fc2.weight)
        self.drop = nn.Dropout(0.2)
        self.bn2 = nn.BatchNorm1d(num_features = hidden_nodes2)
        
        self.fc3 = nn.Linear(hidden_nodes2,output_nodes)
        nn.init.xavier_uniform_(self.fc3.weight)
        
        self.out_act = nn.Sigmoid();
        
    def forward(self,X):
        X = F.relu(self.bn1(self.fc1(X)))
        X = self.fc2(X)
        X = self.drop(X)
        X = F.relu(X)
        X = self.fc3(X)
        # out = self.out_act(X)
        return X
        
    

In [17]:
def file_loader(filepath):
    data = open(filepath)
    feature_list=[] 
    
    for idx,line in enumerate (data):
        feature=line.split()
        feature_list.append(feature[0])
    
    feature_list=np.array(feature_list)
    input_tensor = feature_list[:362]
    input_tensor.resize(1,362)
    input_tensor = input_tensor.astype(np.float)
    input_tensor = torch.from_numpy(input_tensor)
    input_tensor = input_tensor.float()
    
    
    return input_tensor;

In [18]:
data=np.load("../../model_data/data_float360_8_fg_cs.npy")
train_data,test_data = getdataloaders(data,50,5000)
trainloader = torch.from_numpy(train_data)
testloader =  torch.from_numpy(test_data)


In [19]:
model = network()
model.load_state_dict(torch.load('reeds_net360_8_fg_cs.pt'))
start = time.time()
model.eval()
out =model.forward(testloader[1,:47,:].float().t())
end = time.time()
print(end - start)

0.018334388732910156


In [20]:
traced_script_module = torch.jit.trace(model,trainloader[1,:47,:].float().t() )

In [21]:
t1 = time.time()
output = traced_script_module(testloader[1,:47,:].float().t())
print(time.time()-t1)
# print(output)

0.05817747116088867


In [22]:
traced_script_module.save("cpp_model360_8_fg_cs.pt")